In [ ]:
import os
import json
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
train_dir = '../input/aptos2019-blindness-detection/train_images'

df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

df['name'] = df['id_code'].apply(lambda x:x+'.png')

df.diagnosis = df.diagnosis.astype(str)

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.1, random_state = 42, shuffle=True)

from keras.preprocessing.image import ImageDataGenerator

BS = 16
IMG_SIZE = 300
SIZE = (IMG_SIZE,IMG_SIZE)

datagen = ImageDataGenerator(
            preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
            rescale=1/255.0)

train_set=datagen.flow_from_dataframe(dataframe=train,
                                            directory=train_dir,
                                            x_col='name',
                                            y_col='diagnosis',
                                            batch_size=BS,
                                            seed=42,
                                            shuffle=True,
                                            class_mode='categorical',
                                            target_size=SIZE)

val_set=datagen.flow_from_dataframe(dataframe=val,
                                            directory=train_dir,
                                            x_col='name',
                                            y_col='diagnosis',
                                            batch_size=BS,
                                            seed=42,
                                            shuffle=True,
                                            class_mode='categorical',
                                            target_size=SIZE)


In [ ]:
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.models import Sequential
# from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization

    
# base = tf.keras.applications.EfficientNetB3(weights = None,
#                                             include_top=False, 
#                                             input_shape=(IMG_SIZE, IMG_SIZE, 3),
#                                             drop_connect_rate=0.5)
# base.load_weights('../input/weight/efficientnet-b3_noisy-student_notop.h5', by_name=True)

# model = Sequential()

# model.add(base)

# model.add(GlobalAveragePooling2D())
# model.add(Dense(512, activation = 'relu'))
# model.add(BatchNormalization())
# model.add(Dense(512, activation = 'relu'))
# model.add(BatchNormalization())
# model.add(Dense(5, activation = 'softmax'))

# model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), 
#               metrics= ['accuracy'])

# model.summary()

In [ ]:
model = tf.keras.models.load_model('../input/bbbbbbbbbbb/bestmodel (21).h5')

In [ ]:
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# checkpoint = ModelCheckpoint("bestmodel.h5",
#                              save_best_only=True,
#                              monitor='val_loss',
#                              mode='min',
#                              verbose=1)

# reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.2,
#                               patience=3,
#                               min_lr=1e-6,
#                               mode='min',
#                               verbose=1)

In [ ]:
# STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
# STEP_SIZE_VALID = val_set.n//val_set.batch_size

# EPOCHS = 5

# history = model.fit(train_set,
#                     validation_data=val_set,
#                     epochs=EPOCHS,
#                     batch_size=BS,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_steps=STEP_SIZE_VALID,
#                     callbacks=[checkpoint, reduce_lr])

# model.save('model.h5')

In [ ]:
sub = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
test_dir = '../input/aptos2019-blindness-detection/test_images/'

results = []

for img_name in sub['id_code']:
    img = cv2.imread(test_dir + img_name + '.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.array(img)
    img = img / 255.0
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 3)
    prediction = model.predict(img)
    prediction = prediction[0]
    results.append(np.argmax(prediction, axis = 0))

In [ ]:
pred_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
pred_df['diagnosis'] = results
pred_df.to_csv('submission.csv',index=False)
pred_df